<a href="https://colab.research.google.com/github/liorZucker11/cloud-computing/blob/main/Shablool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and pips:

In [312]:
#!pip install ipywidgets
#!pip install IPython
#!pip install firebase
#!pip install nltk

#-------Settting the Text To Image---------
# Check the GPU
#!nvidia-smi
# Install dependencies

#!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch
#%pip install git+https://github.com/crowsonkb/k-diffusion
# Download the diffusion model
# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda

#!mkdir v-diffusion-pytorch/checkpoints
#!curl -L --http1.1 https://the-eye.eu/public/AI/models/v-diffusion/cc12m_1_cfg.pth > v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth

In [313]:
from IPython.display import display, Image
import ipywidgets as widgets
from firebase import firebase
from nltk.stem import PorterStemmer
import re
import os
import urllib.request
import requests
import time

#import of ChatBot
import nltk
from nltk.chat.util import Chat, reflections

#import of Text To Image
import gc
import math
import sys

import clip
import k_diffusion as K
import torch
from torch import nn
from torchvision import utils
from torchvision.transforms import functional as TF
from tqdm.notebook import trange, tqdm

sys.path.append('/content/v-diffusion-pytorch')

from diffusion import get_model


# **DB** - firebase:

In [314]:
firebase = firebase.FirebaseApplication('https://cloudproject-shablool-default-rtdb.europe-west1.firebasedatabase.app/', None)

"""
  Retrieves 'links' and 'links_counter' data associated with the given key from Firebase index.
  Returns a transposed list or None if the key doesn't exist.
"""
def getKey(key):
  word_data = firebase.get(f'/index/{key}',None)
  if word_data == None:
    return word_data
  tmp = [word_data['links'],word_data['links_counter']]
  transposed = [[row[i] for row in tmp] for i in range(len(tmp[0]))]
  return transposed

#Updates or adds a key-value pair in the Firebase index.
def updateKeyValue(key,value):
  firebase.put("/index/", key, value)

# Deletes a key and all associated values from the Firebase index.
def deleteKey(key):
  firebase.delete('/index', key)

def get_key_with_highest_count():
    # Query to order by 'count' under each word and get the last one (the highest)
    query_result = firebase.get('/index', None, params={'orderBy': '"count"', 'limitToLast': 1})
    if not query_result:
        return "No data found.", 0
    word = list(query_result.keys())[0]
    count = query_result[word]['count']
    return word, count

def get_key_with_lowest_count():
    # Query to order by 'count' under each word and get the first one (the lowest)
    query_result = firebase.get('/index', None, params={'orderBy': '"count"', 'limitToFirst': 1})
    if not query_result:
        return "No data found.", 0
    word = list(query_result.keys())[0]
    count = query_result[word]['count']
    return word, count

# Variables and Functions

##Logo and variables:

In [315]:
#Initial Global Variables
container=widgets.Box()
results_output=widgets.Output()

In [316]:
# Corrected URL of the logo stored in GitHub (raw content URL)
logo_url = 'https://raw.githubusercontent.com/liorZucker11/cloud-computing/main/logo.jpg'  # Update this URL

# Download the logo
logo_response = requests.get(logo_url)

logo = widgets.Image()
# Ensure the response was successful
if logo_response.status_code == 200:
    logo = widgets.Image(value=logo_response.content, format='jpg', layout=widgets.Layout(height='100px', width='auto', margin='0 0 20px 0'))
else:
    print("Failed to download the logo.")

# Define the layout for the logo with align_items='center'
logo_layout = widgets.Layout(align_items='center', max_width='200px')
# Apply the logo_layout to the logo widget
logo.layout = logo_layout


## Functions for searching algorithem:

In [317]:
#first level
def sort_links_by_counter(links):
    # Sort the list of lists based on the second element (counter) in each sublist
    sorted_links = sorted(links, key=lambda x: x[1], reverse=True)
    # Extract the first element (link) from each sublist and return as a 1-dimensional list
    return [link[0] for link in sorted_links]


#second level
def filter_common_links(big_data_structure):
    # Step 1: Find common links across all wordDatas
    common_links = set(link for link, _ in big_data_structure[0])  # Initialize with links from the first wordData
    for wordData in big_data_structure[1:]:  # Start from the second wordData
        current_links = set(link for link, _ in wordData)
        common_links.intersection_update(current_links)  # Keep only links that are also in the current wordData
    # Step 2: Filter each wordData to keep only elements with links in common_links
    filtered_data_structure = []
    for wordData in big_data_structure:
        filtered_wordData = [[link, counter] for link, counter in wordData if link in common_links]
        filtered_data_structure.append(filtered_wordData)
    return filtered_data_structure


#third level
def calculate_average_counters(filtered_data_structure):
    link_counters = {}  # Dictionary to hold the total counters for each link and the number of occurrences
    # Aggregate counters for each link
    for wordData in filtered_data_structure:
        for link, counter in wordData:
            if link in link_counters:
                link_counters[link][0] += counter  # Add to the total counter
                link_counters[link][1] += 1  # Increment occurrences
            else:
                link_counters[link] = [counter, 1]  # Initialize with the first counter and set occurrences to 1
    # Calculate average for each link and prepare the result list
    average_counters = [[link, total_counter / occurrences] for link, (total_counter, occurrences) in link_counters.items()]
    # Sort the result list by link for consistent output, if needed
    average_counters.sort(key=lambda x: x[0])
    return average_counters

#fourth level
def sort_by_counters_desc(average_counters):
    # Sort the list by the counter value in descending order
    sorted_average_counters = sorted(average_counters, key=lambda x: x[1], reverse=True)
    return sorted_average_counters

#fivth level
def extract_links(average_counters):
    links_only = [link for link, _ in average_counters]
    return links_only

"""
  Searches for links associated with the stemmed words in the query.

  This function takes a search query as input, stems the words using Porter Stemmer,
  retrieves data for each stemmed word from the Firebase index using the getKey function,
  filters common links across all search results, calculates the average counters for each link,
  sorts the links by their counters in descending order, and finally returns a list of links
  extracted from the sorted results.

  If no results are found for the query, it returns a message indicating that the key
  does not exist in the index.
"""
def search(query):
    stemmer = PorterStemmer()
    query = query.lower()
    results = []
    query_words = query.split()
    for word in query_words:
      word = stemmer.stem(word)
      results.append(getKey(word))
    if results == [] or results == [""] or results == [None]:
      return None
    print(results)
    level1 = filter_common_links(results)
    level2 = calculate_average_counters(level1)
    level3 = sort_by_counters_desc(level2)
    level4 = extract_links(level3)
    return level4

## Functions for admin screen algo:

In [318]:
 # Processes the input value to prepare it for indexing.
def prepare_value_for_index(value):
  #https://portal.azure.com/ 1
  count,links,links_counter = process_data(value)
  val = {"count" : count , "links": links, "links_counter":links_counter}
  return val

    # Processes the input string to extract site URLs and their corresponding numbers.
def process_data(input_string):
  lines = input_string.split('\n')
  sites = []
  numbers = []
  for line in lines:
        site, number = line.strip().split()
        sites.append(site)
        numbers.append(int(number))
  return sum(numbers), sites, numbers

    # Checks if the input value is in the correct format for updating.
def check_Value_to_update(value):
  #https://portal.azure.com/ 1
  pattern = r"^.+ \d+$"
  lines = value.split('\n')
  for line in lines:
      if not re.match(pattern, line.strip()):
          return False
  return True

    # Checks the syntax of the input key.
    # The key syntax is considered invalid if it contains any of the forbidden characters: '.', ',', '$', '#', '[', ']', '/'.
def check_key_syntax(key):
  forbidden_chars = ['.', ',', '$', '#', '[', ']', '/']
  for char in key:
      if char in forbidden_chars:
          return False
  return True
admin_stemmer = PorterStemmer()

#**Screens**

#**Admin Screen:**

In [319]:
def create_admin_screen():
  def on_return_button_clicked(b):
      container.children = []
      container.layout = widgets.Layout()
      Main_Screen()
  # Input text box
  input_text = widgets.Text(
    placeholder='Enter text here...',
    description='Key:',
    disabled=False,
    layout=widgets.Layout(width='100%', margin='0 0 10px 0'),
    style={'description_width': 'initial', 'border': '1px solid black'}
  )

  # Larger text area for displaying results
  results_text = widgets.Textarea(
    placeholder='Results will be shown here...',
    description='Results:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px', margin='0 0 10px 0'),
    style={'description_width': 'initial', 'border': '1px solid black'}
  )

  # checkbox to know if the admin want to use stemmer or not
  checkbox = widgets.Checkbox(
    value=False,
    description='use stem?',
    disabled=False
  )

  # Buttons for database operations
  delete_button = widgets.Button(description="Delete", button_style='danger', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  edit_button = widgets.Button(description="Edit", disabled=True, layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  add_button = widgets.Button(description="Add", button_style='success', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  get_button = widgets.Button(description="Get", button_style='info', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px'))

  # Reset spesific labels to ""
  def reset_labels(num):
    if num == 1:
      comment_string.value = ""
      stem_string.value = ""
    if num == 2:
      comment_string.value = ""
      stem_string.value = ""
      results_text.value = ''
    if num == 3:
      results_text.value = ''
    if input_text.value == None or input_text.value == "":
        return 1
    return 0

  # Function to handle return button click
  def on_return_button_clicked(b):
    container.children = []
    container.layout = widgets.Layout()
    Main_Screen()

  # Actions for buttons
  #action for delete button
  def on_delete_button_clicked(b):
    value = input_text.value
    if reset_labels(1) == 1:
      return
    if checkbox.value == True:
      value = admin_stemmer.stem(input_text.value)
      stem_string.value = f"key after stem: {value}"
    data = getKey(value)
    if data == None:
      comment_string.value = "you cant delete this key becuase it already not there!"
      return
    deleteKey(input_text.value)
    comment_string.value = "delete successful"
    reset_labels(3)

  #action for edit button
  def on_edit_button_clicked(b):
    if reset_labels(1) == 1:
      return
    if results_text.value == None or results_text.value == "":
      comment_string.value =  "you cant edit a key without a value. value for example:https://portal.azure.com/ 1"
      return
    if check_key_syntax(input_text.value) == False:
      comment_string.value =  "['.', ',', '$', '#', '[', ']', '/'] chars cannot be in the key name"
      return
    if(check_Value_to_update(results_text.value) == False):
      comment_string.value =  "value not in the structure like it should. value for example:https://portal.azure.com/ 1"
      return
    val_for_index = prepare_value_for_index(results_text.value)
    updateKeyValue(input_text.value,val_for_index)
    comment_string.value = "update successful"

  #action for add button
  def on_add_button_clicked(b):
    if reset_labels(1) == 1:
      return
    if results_text.value == None or results_text.value == "":
      comment_string.value =  "you cant add a key without a value. value for example:https://portal.azure.com/ 1"
      return
    value = input_text.value
    if checkbox.value == True:
      value = admin_stemmer.stem(input_text.value)
      stem_string.value = f"key after stem: {value}"
    data = getKey(value)
    if data != None:
      comment_string.value =  "you cant add this key beacuse it already in the index!"
      return
    if check_key_syntax(input_text.value) == False:
      comment_string.value =  "['.', ',', '$', '#', '[', ']', '/'] chars cannot be in the key name"
      return
    if(check_Value_to_update(results_text.value) == False):
      comment_string.value =  "value not in the structure like it should. value for example:https://portal.azure.com/ 1"
      return
    val_for_index = prepare_value_for_index(results_text.value)
    updateKeyValue(input_text.value,val_for_index)
    comment_string.value = "add successful"

  #action for get button
  def on_get_button_clicked(b):
    if reset_labels(2) == 1:
      return
    edit_button.disabled = False
    value = input_text.value
    if checkbox.value == True:
      value = admin_stemmer.stem(input_text.value)
      stem_string.value = f"key after stem: {value}"
    data = getKey(value)
    if data == None:
      comment_string.value = "The key does not exist in the index."
    else:
      results_text.value = '\n'.join([' '.join(map(str, sub_list)) for sub_list in data])


  # Attach actions to buttons
  delete_button.on_click(on_delete_button_clicked)
  edit_button.on_click(on_edit_button_clicked)
  add_button.on_click(on_add_button_clicked)
  get_button.on_click(on_get_button_clicked)
  return_button.on_click(on_return_button_clicked)


  # Arrange buttons horizontally
  buttons_hbox = widgets.HBox([get_button, add_button, edit_button, delete_button], layout=widgets.Layout(justify_content='flex-start'))

  comment_string = widgets.Label("")
  stem_string = widgets.Label("")
  stem_hbox = widgets.HBox([checkbox, stem_string], layout=widgets.Layout(justify_content='flex-start'))

  # Arrange all widgets vertically
  additional_string = widgets.Label("Admin Screen")
  hBox_layout = widgets.Layout(display='flex', justify_content='space-between', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
  hBox = widgets.HBox(children=[return_button,additional_string ,logo], layout=hBox_layout)

  vBox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
  vBox = widgets.VBox([input_text,comment_string ,results_text, buttons_hbox,stem_hbox], layout=vBox_layout)

  container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', justify_content='flex-start', height='100vh')
  container = widgets.Box(children=[hBox, vBox], layout=container_layout)

  # Display everything
  display(container)


#**Statistics Screen:**

In [320]:
def create_statistics_screen():

  def on_return_button_clicked(b):
    container.children = []
    container.layout = widgets.Layout()
    Main_Screen()

  word_with_highest_count, count_high = get_key_with_highest_count()
  most_Freq = widgets.Label(f"Most Frequent Word: '{word_with_highest_count}' with a count of {count_high}")
  word_with_lowest_count, count_low = get_key_with_lowest_count()
  less_Freq = widgets.Label(f"Least Frequent Word:  '{word_with_lowest_count}' with a count of {count_low}")
  most_search_Freq = widgets.Label("Most Search Frequent Word: Azure")

  input_text = widgets.Text(
    placeholder='If you want to know more enter a word...',
    description='Word:',
    disabled=False,
    layout=widgets.Layout(width='95%', margin='10px 10px 10px 10px'),
    style={'description_width': 'initial', 'border': '1px solid black'}
  )

    # Larger text area for displaying results
  results_text = widgets.Textarea(
      placeholder='Results will be shown here...',
      description='Result:',
      disabled=False,
      layout=widgets.Layout(width='100%', height='200px', margin='0 0 10px 0'),
      style={'description_width': 'initial', 'border': '1px solid black'}
  )

  return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px', alignment = 'left'))
  return_button.on_click(on_return_button_clicked)

  vbox_layout = widgets.Layout(display='flex', flex_flow='column', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
  vbox = widgets.VBox([return_button, logo,  most_Freq, less_Freq, most_search_Freq,input_text,results_text], layout=vbox_layout)
  container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh')
  container = widgets.Box(children=[vbox], layout=container_layout)

  # Display the elements
  display(container)

# **The ChatBot**

In [321]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatBot.', 'I go by the name ChatBot.']),
    (r'(.*) your name?', ['You can call me ChatBot.', 'I go by the name ChatBot.']),
    (r'what is Microsoft Azure?', ['Microsoft Azure is a cloud computing platform by Microsoft.']),
    (r'(.*) cloud computing(.*)', ['Cloud computing is a technology that allows users to access and use computing resources over the internet.']),
    (r'Azure services', ['Microsoft Azure offers a variety of services, including compute, storage, databases, and more.']),
    (r'virtual machines', ['Azure Virtual Machines allow you to run Windows or Linux virtual machines in the cloud.']),
    (r'web development on Azure', ['Azure provides services for web development, including Azure App Service and Azure Functions.']),
    (r'Azure App Service', ['Azure App Service is a fully managed platform for building, deploying, and scaling web apps.']),
    (r'Azure Functions', ['Azure Functions is a serverless compute service that enables you to run event-triggered code without explicitly provisioning infrastructure.'])
    # You can add more patterns and responses here
]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#**ChatBot Screen:**

In [329]:
def create_chatbot_screen():
    chatbot = Chat(patterns, reflections)
    return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px'))

    # Create input text widget for user input
    input_text = widgets.Text(
        value='',
        placeholder='Type your message here',
        description='You:',
        disabled=False
    )

    output = widgets.Output(layout={'border': '1px solid black', 'width': '100%'})


    # Function to handle return button click
    def on_return_button_clicked(b):
      container.children = []
      container.layout = widgets.Layout()
      Main_Screen()


    def on_submit(change):
        user_input = input_text.value
        response = chatbot.respond(user_input)
        if response is None:
            response = "I'm not sure how to respond to that."

        with output:
            print(f"You: {user_input}")
            print(f"ChatBot: {response}")
        time.sleep(1)
        input_text.value = ''

    input_text.on_submit(on_submit)
    return_button.on_click(on_return_button_clicked)

    vbox_layout = widgets.Layout(display='flex', flex_flow='column', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px',overflow='hidden')
    vbox = widgets.VBox([return_button, input_text, output], layout=vbox_layout)
    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh',overflow='hidden')
    container = widgets.Box(children=[vbox], layout=container_layout)

    display(container)

# **The Text To Image**

In [323]:
inner_model = get_model('cc12m_1_cfg')()
_, side_y, side_x = inner_model.shape
inner_model.load_state_dict(torch.load('v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth', map_location='cpu'))
inner_model = inner_model.half().cuda().eval().requires_grad_(False)
model = K.external.VDenoiser(inner_model)
clip_model = clip.load(inner_model.clip_model, jit=False, device='cpu')[0]

#hyper parameters
prompt = ""
weight = 5
n_images = 1
steps =   50
seed = 0
display_every =   10

target_embed = clip_model.encode_text(clip.tokenize(prompt)).float().cuda()


class CFGDenoiser(nn.Module):
    def __init__(self, model, cond_scale):
        super().__init__()
        self.inner_model = model
        self.cond_scale = cond_scale

    def forward(self, x, sigma, clip_embed):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        clip_embed_in = torch.cat([torch.zeros_like(clip_embed), clip_embed])
        uncond, cond = self.inner_model(x_in, sigma_in, clip_embed=clip_embed_in).chunk(2)
        return uncond + (cond - uncond) * self.cond_scale


def callback(info):
    if info['i'] % display_every == 0:
        nrow = math.ceil(info['denoised'].shape[0] ** 0.5)
        grid = utils.make_grid(info['denoised'], nrow, padding=0)
        tqdm.write(f'Step {info["i"]} of {steps}, sigma {info["sigma"]:g}:')
        display(K.utils.to_pil_image(grid))
        tqdm.write(f'')


def generate_image(prompt):
    target_embed = clip_model.encode_text(clip.tokenize(prompt)).float().cuda()
    gc.collect()
    torch.cuda.empty_cache()
    torch.manual_seed(seed)
    sigmas = K.sampling.get_sigmas_karras(steps, 1e-2, 160, device='cuda')
    x = torch.randn([n_images, 3, side_y, side_x], device='cuda') * sigmas[0]
    model_wrap = CFGDenoiser(torch.cuda.amp.autocast()(model), weight)
    extra_args = {'clip_embed': target_embed.repeat([n_images, 1])}
    outs = K.sampling.sample_lms(model_wrap, x, sigmas, extra_args=extra_args)
    tqdm.write('Done!')
    img = K.utils.to_pil_image(outs[-1])
    display(img)


#**Text To Image Screen**

In [324]:
def create_texttoimage_screen():
    # Consider moving model loading to a separate initialization section if applicable

    prompt_label = widgets.Label('Enter your prompt:')
    prompt_input = widgets.Text(value='', placeholder='Type your prompt here', description='', disabled=False)
    prompt = prompt_input.value

    # Parameters for image generation
    steps_input = widgets.IntText(value=50, description='Steps:', disabled=False)
    seed_input = widgets.IntText(value=0, description='Seed:', disabled=False)
    weight_input = widgets.FloatText(value=7.5, description='Weight:', disabled=False)
    generate_button = widgets.Button(description='Generate Image')
    status_label = widgets.Label()  # For showing status/progress
    output_area = widgets.Output()
    return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px'))

    # Function to handle return button click
    def on_return_button_clicked(b):
        container.children = []
        container.layout = widgets.Layout()
        Main_Screen()


    def on_generate_button_clicked(b):
        current_prompt = prompt_input.value  # Get the current value of the prompt input
        status_label.value = "Generating image... Please wait."
        generate_image(current_prompt)  # Pass the current prompt to the function

    generate_button.on_click(on_generate_button_clicked)
    return_button.on_click(on_return_button_clicked)

    # Layout for the text-to-image screen
    form_items = [
        return_button,
        prompt_label,
        prompt_input,
        steps_input,
        seed_input,
        weight_input,
        generate_button,
        status_label,  # Add this to the UI
        output_area
    ]

    vbox_layout = widgets.Layout(display='flex', flex_flow='column', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
    vbox = widgets.VBox(form_items, layout=vbox_layout)
    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh')
    container = widgets.Box(children=[vbox], layout=container_layout)
    display(container)

#**Search Screen (main):**

In [325]:
def Main_Screen():
    # Button actions
    def statistics_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_statistics_screen()

    def admin_screen_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_admin_screen()

    def chatbot_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_chatbot_screen()

    def texttoimage_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_texttoimage_screen()

    # Create buttons with actions
    admin_screen_button = widgets.Button(description="Admin Screen", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    statistics_button = widgets.Button(description="Statistics", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    chatbot_button = widgets.Button(description="ChatBot", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    texttoimage_button = widgets.Button(description="Text To Image", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))

    admin_screen_button.on_click(admin_screen_action)
    statistics_button.on_click(statistics_action)
    chatbot_button.on_click(chatbot_action)
    texttoimage_button.on_click(texttoimage_action)
    # Adjust layout for input field, button, and results output
    input_layout = widgets.Layout(width='50%', justify_content='center')
    button_layout = widgets.Layout(width='50%', justify_content='center', margin='10px 0 0 0')
    results_layout = widgets.Layout(border='1px solid black', border_radius='5px', width='90%', margin='20px 0', padding='10px', visibility='hidden')
    # Create the search text box, button, and results output
    search_input = widgets.Text(value='', placeholder='Type here to search', description='', disabled=False, layout=input_layout)
    search_button = widgets.Button(description='SEARCH', button_style='success', tooltip='Click to search', icon='search', layout=button_layout)
    results_output = widgets.Textarea(layout=results_layout, rows=10)

    # Function to simulate a search operation and display results
    def perform_search(query):
      results_output.layout.visibility = 'visible'
      if search_input.value == None or search_input.value == "":
        results_output.value = ''
        return
      resultsLinks = search(search_input.value)
      if resultsLinks == None:
        results_output.value = "The key does not exist in the index."
      else:
        results_output.value = '\n'.join(resultsLinks)

    search_button.on_click(perform_search)

    # Organize the layout
    top_buttons = widgets.HBox([admin_screen_button, statistics_button, chatbot_button, texttoimage_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))
    vbox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
    vbox = widgets.VBox([top_buttons, logo, search_input, search_button, results_output], layout=vbox_layout)
    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh')
    container = widgets.Box(children=[vbox], layout=container_layout)

    # Display the elements
    display(container)


# **The Website**

In [330]:
Main_Screen()

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(VBox(children=(Button(button_style='warning', description='Return', layout=Layout(height='50px',…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(VBox(children=(Button(button_style='warning', description='Return', layout=Layout(height='50px',…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

...

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(VBox(children=(Button(button_style='warning', description='Return', layout=Layout(height='50px',…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(HBox(children=(Button(button_style='warning', description='Return', layout=Layout(height='50px',…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …